In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.kernel_approximation import RBFSampler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, make_scorer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import learning_curve
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC # not stochastic
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import export_graphviz
import re
from sqlalchemy import create_engine
import pickle
import warnings
# import requests
# from bs4 import BeautifulSoup
# import time
# from fake_useragent import UserAgent
# import sys, os
import flask
import json
from collections import defaultdict

warnings.filterwarnings("ignore")
%matplotlib inline

RANDOM_STATE = 1
TEST_SIZE = 0.3

### Make arbitrary data

In [2]:
with open("cleaned2_data.pkl", 'rb') as picklefile: 
    df = pickle.load(picklefile)
X_cols = [col for col in df.columns if col not in ['q_id', 'q_user', 'a_id', 'a_user', 'label_col']]
X = df[X_cols]
y = df['label_col']

In [11]:
rf = RandomForestClassifier(n_estimators = 1, max_depth = 1)
rf.fit(X, y)
one_tree = rf[0].tree_

In [12]:
#export_graphviz(one_tree)

In [13]:
one_tree.children_left

array([ 1, -1, -1])

In [14]:
one_tree.children_right

array([ 2, -1, -1])

In [15]:
one_tree.feature

array([29, -2, -2])

In [16]:
one_tree.threshold

array([ 3.20590925, -2.        , -2.        ])

### Turn randon forest’s one tree object into a nested list so json can use it

In [17]:
def tree_to_li(one_tree, parent_index = None, direction = None):
    node = {}
    if direction == "l":
        node['name'] = one_tree.children_left[parent_index]
    elif direction == "r":
        node['name'] = one_tree.children_right[parent_index]
    else:
        node['name'] = 0
    node['feat'] = one_tree.feature[node['name']]
    node['thres'] = one_tree.threshold[node['name']]
    if node['name'] != -1:
        node['children'] = [tree_to_li(one_tree, node['name'], 'l'), 
                            tree_to_li(one_tree, node['name'], 'r')] 
    return node

In [18]:
tree_to_li(one_tree)

{'children': [{'children': [{'feat': -2, 'name': -1, 'thres': -2.0},
    {'feat': -2, 'name': -1, 'thres': -2.0}],
   'feat': -2,
   'name': 1,
   'thres': -2.0},
  {'children': [{'feat': -2, 'name': -1, 'thres': -2.0},
    {'feat': -2, 'name': -1, 'thres': -2.0}],
   'feat': -2,
   'name': 2,
   'thres': -2.0}],
 'feat': 29,
 'name': 0,
 'thres': 3.205909252166748}

## Stuff I ended up not using

In [ ]:
# Source: sf17_ds6/class_lectures/week01-benson/03-python_jupyter/Python2.ipynb
# def tree(): 
#     return defaultdict(tree)

In [ ]:
# Source: http://scikit-learn.org/stable/auto_examples/tree/plot_unveil_tree_structure.html
# The tree structure can be traversed to compute various properties such
# as the depth of each node and whether or not it is a leaf.
# node_depth = np.zeros(shape=n_nodes)
# is_leaves = np.zeros(shape=n_nodes, dtype=bool)
# stack = [(0, -1)]  # seed is the root node id and its parent depth
# while len(stack) > 0:
#     node_id, parent_depth = stack.pop()
#     node_depth[node_id] = parent_depth + 1

#     # If we have a test node
#     if (children_left[node_id] != children_right[node_id]):
#         stack.append((children_left[node_id], parent_depth + 1))
#         stack.append((children_right[node_id], parent_depth + 1))
#     else:
#         is_leaves[node_id] = True